In [22]:
#data preprocessing
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC
#displayd data
from IPython.display import display
%matplotlib inline

In [56]:
# Read data and drop redundant column.
data = pd.read_csv('Data/final_processed_1.csv')
display(data.head())


# Total number of matches.
n_matches = data.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.FTR == 'H'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100


# Print the results
print ("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print ("Number of matches won by home team: {}".format(n_homewins))
print ("Win rate of home team: {:.2f}%".format(win_rate))

,div,key,team1,team2,date,FTR,team1_home_win,team1_home_draw,team1_away_win,team1_away_draw,...,BbMx_2.5,BbAv_2.5,BbMx_2.5.1,BbAv_2.5.1,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,0,Arsenal_West Brom_2008-08-16,Arsenal,West Brom,2008-08-16,H,1.0,0.0,0.0,0.0,...,1.71,1.65,2.25,2.14,26.0,-1.5,1.88,1.81,2.12,1.97
1,1,Bolton_Stoke_2008-08-16,Bolton,Stoke,2008-08-16,H,1.0,0.0,0.0,0.0,...,2.33,2.16,1.70,1.64,27.0,0.0,1.35,1.31,3.56,3.27
2,2,Everton_Blackburn_2008-08-16,Everton,Blackburn,2008-08-16,A,0.0,0.0,0.0,0.0,...,2.34,2.17,1.70,1.63,28.0,0.0,1.73,1.43,3.05,2.76
3,3,Hull_Fulham_2008-08-16,Hull,Fulham,2008-08-16,H,1.0,0.0,0.0,0.0,...,2.32,2.15,1.69,1.64,26.0,0.0,1.91,1.85,2.07,1.94
4,4,Middlesbrough_Tottenham_2008-08-16,Middlesbrough,Tottenham,2008-08-16,H,1.0,0.0,0.0,0.0,...,1.90,1.81,2.12,1.92,27.0,0.0,2.45,2.30,1.65,1.58


Total number of matches: 3040
Number of features: 91
Number of matches won by home team: 1390
Win rate of home team: 45.72%


In [57]:
# Separate into feature set and target variable
#FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
X_all =data.drop(['div','FTR', 'key', 'date', 'WHH','WHD','WHA'], axis=1)  # df.columns is zero-based pd.Index
# X_all =X_all.drop(0, axis=1)  # df.columns is zero-based pd.Index 

y_all = data['FTR']

# homeTeams = data['team1']
# awayTeams = data['team2']
# teamIds ={}
# id=1
# for team in homeTeams:
#     if team not in teamIds:
#         teamIds[team] = id
#         id +=1
        
# for team in awayTeams:
#     if team not in teamIds:
#         teamIds[team] = id
#         id +=1
        
# for index,row in X_all.iterrows():
#     row['team1'] = teamIds[row['team1']]
#     row['team2'] = teamIds[row['team2']]
#     X_all.loc[index] = row
    
# print(teamIds)
display(X_all.head())

{'Arsenal': 1, 'Bolton': 2, 'Everton': 3, 'Hull': 4, 'Middlesbrough': 5, 'Sunderland': 6, 'West Ham': 7, 'Aston Villa': 8, 'Chelsea': 9, 'Man United': 10, 'Blackburn': 11, 'Fulham': 12, 'Liverpool': 13, 'Newcastle': 14, 'Stoke': 15, 'Tottenham': 16, 'West Brom': 17, 'Man City': 18, 'Wigan': 19, 'Portsmouth': 20, 'Wolves': 21, 'Birmingham': 22, 'Burnley': 23, 'Blackpool': 24, 'QPR': 25, 'Swansea': 26, 'Norwich': 27, 'Reading': 28, 'Southampton': 29, 'Crystal Palace': 30, 'Cardiff': 31, 'Leicester': 32, 'Bournemouth': 33, 'Watford': 34}


,team1,team2,team1_home_win,team1_home_draw,team1_away_win,team1_away_draw,team1_home_goals,team1_away_goals,team1_home_goals_conceded,team1_away_goals_conceded,...,BbMx_2.5,BbAv_2.5,BbMx_2.5.1,BbAv_2.5.1,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,1,17,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.71,1.65,2.25,2.14,26.0,-1.5,1.88,1.81,2.12,1.97
1,2,15,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,...,2.33,2.16,1.70,1.64,27.0,0.0,1.35,1.31,3.56,3.27
2,3,11,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,...,2.34,2.17,1.70,1.63,28.0,0.0,1.73,1.43,3.05,2.76
3,4,12,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,...,2.32,2.15,1.69,1.64,26.0,0.0,1.91,1.85,2.07,1.94
4,5,16,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,...,1.90,1.81,2.12,1.92,27.0,0.0,2.45,2.30,1.65,1.58


In [58]:
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

# Standardising the data.
from sklearn.preprocessing import scale
# X_all = X_all.reset_index()
# X_all = X_all[X_all.label=='desired_one']
#Center to the mean and component wise scale to unit variance.
cols = X_all.columns

for col in cols:
    X_all[col] = scale(X_all[col])
X_all = preprocess_features(X_all)
# X_all.ha

/home/shravan/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [59]:
##splitting
from sklearn.model_selection import train_test_split
# train_test_split(X, y, test_size=0.2, shuffle=False)
# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.2,
                                                    shuffle = False)

In [60]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    
    return 0,sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print (f1, acc)
    print ("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print ("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [61]:


# Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print ('')

train_predict(clf_B, X_train, y_train, X_test, y_test)
print ('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print ('')



Training a LogisticRegression using a training set size of 2432. . .
Trained model in 0.2711 seconds
Made predictions in 0.0006 seconds.
0 0.662828947368421
F1 score and accuracy score for training set: 0.0000 , 0.6628.
Made predictions in 0.0003 seconds.
F1 score and accuracy score for test set: 0.0000 , 0.4753.

Training a SVC using a training set size of 2432. . .
Trained model in 1.0087 seconds
Made predictions in 0.6865 seconds.
0 0.6772203947368421
F1 score and accuracy score for training set: 0.0000 , 0.6772.
Made predictions in 0.1709 seconds.
F1 score and accuracy score for test set: 0.0000 , 0.5773.

Training a XGBClassifier using a training set size of 2432. . .
Trained model in 2.3830 seconds
Made predictions in 0.0315 seconds.
0 0.7446546052631579
F1 score and accuracy score for training set: 0.0000 , 0.7447.
Made predictions in 0.0087 seconds.
F1 score and accuracy score for test set: 0.0000 , 0.6053.



/home/shravan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shravan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
